# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 2 2022 9:43AM,238619,15,ALIFBA16QSGCRKR,"Alliance Pharma, Inc.",Released
1,Jun 2 2022 9:43AM,238619,15,30009093,"Alliance Pharma, Inc.",Released
2,Jun 2 2022 9:43AM,238619,15,30009096,"Alliance Pharma, Inc.",Released
3,Jun 2 2022 9:43AM,238619,15,30009098,"Alliance Pharma, Inc.",Released
4,Jun 2 2022 9:43AM,238619,15,30009101,"Alliance Pharma, Inc.",Released
5,Jun 2 2022 9:43AM,238619,15,30009097,"Alliance Pharma, Inc.",Released
6,Jun 2 2022 9:43AM,238619,15,30009102,"Alliance Pharma, Inc.",Released
7,Jun 2 2022 9:43AM,238619,15,30009100,"Alliance Pharma, Inc.",Released
8,Jun 2 2022 9:43AM,238619,15,30009105,"Alliance Pharma, Inc.",Released
9,Jun 2 2022 9:43AM,238619,15,30009099,"Alliance Pharma, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
30,238615,Released,1
31,238616,Released,1
32,238617,Executing,1
33,238618,Released,1
34,238619,Released,11


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
238615,NaN,1.0
238616,NaN,1.0
238617,1.0,NaN
238618,NaN,1.0
238619,NaN,11.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
238524,0.0,1.0
238526,0.0,1.0
238529,0.0,60.0
238544,0.0,21.0
238551,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
238524,0,1
238526,0,1
238529,0,60
238544,0,21
238551,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,238524,0,1
1,238526,0,1
2,238529,0,60
3,238544,0,21
4,238551,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,238524,,1
1,238526,,1
2,238529,,60
3,238544,,21
4,238551,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 2 2022 9:43AM,238619,15,"Alliance Pharma, Inc."
11,Jun 2 2022 9:31AM,238618,20,"Andelyn Biosciences, Inc."
12,Jun 2 2022 9:27AM,238617,10,"Senseonics, Incorporated"
13,Jun 2 2022 9:24AM,238616,20,Sartorius Stedim North America
14,Jun 2 2022 9:23AM,238615,12,Hush Hush
15,Jun 2 2022 9:22AM,238613,12,Hush Hush
18,Jun 2 2022 9:12AM,238612,20,"Exact-Rx, Inc."
21,Jun 2 2022 9:06AM,238611,20,Alumier Labs Inc.
40,Jun 2 2022 8:47AM,238610,10,"Amcyte Pharma, Inc."
42,Jun 2 2022 8:46AM,238609,10,Beach Products Inc


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jun 2 2022 9:43AM,238619,15,"Alliance Pharma, Inc.",,11
1,Jun 2 2022 9:31AM,238618,20,"Andelyn Biosciences, Inc.",,1
2,Jun 2 2022 9:27AM,238617,10,"Senseonics, Incorporated",1,
3,Jun 2 2022 9:24AM,238616,20,Sartorius Stedim North America,,1
4,Jun 2 2022 9:23AM,238615,12,Hush Hush,,1
5,Jun 2 2022 9:22AM,238613,12,Hush Hush,,3
6,Jun 2 2022 9:12AM,238612,20,"Exact-Rx, Inc.",,3
7,Jun 2 2022 9:06AM,238611,20,Alumier Labs Inc.,3,16
8,Jun 2 2022 8:47AM,238610,10,"Amcyte Pharma, Inc.",,2
9,Jun 2 2022 8:46AM,238609,10,Beach Products Inc,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 2 2022 9:43AM,238619,15,"Alliance Pharma, Inc.",11,
1,Jun 2 2022 9:31AM,238618,20,"Andelyn Biosciences, Inc.",1,
2,Jun 2 2022 9:27AM,238617,10,"Senseonics, Incorporated",,1
3,Jun 2 2022 9:24AM,238616,20,Sartorius Stedim North America,1,
4,Jun 2 2022 9:23AM,238615,12,Hush Hush,1,
5,Jun 2 2022 9:22AM,238613,12,Hush Hush,3,
6,Jun 2 2022 9:12AM,238612,20,"Exact-Rx, Inc.",3,
7,Jun 2 2022 9:06AM,238611,20,Alumier Labs Inc.,16,3
8,Jun 2 2022 8:47AM,238610,10,"Amcyte Pharma, Inc.",2,
9,Jun 2 2022 8:46AM,238609,10,Beach Products Inc,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 2 2022 9:43AM,238619,15,"Alliance Pharma, Inc.",11,
1,Jun 2 2022 9:31AM,238618,20,"Andelyn Biosciences, Inc.",1,
2,Jun 2 2022 9:27AM,238617,10,"Senseonics, Incorporated",,1
3,Jun 2 2022 9:24AM,238616,20,Sartorius Stedim North America,1,
4,Jun 2 2022 9:23AM,238615,12,Hush Hush,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jun 2 2022 9:43AM,238619,15,"Alliance Pharma, Inc.",11.0,NaN
1,Jun 2 2022 9:31AM,238618,20,"Andelyn Biosciences, Inc.",1.0,NaN
2,Jun 2 2022 9:27AM,238617,10,"Senseonics, Incorporated",NaN,1.0
3,Jun 2 2022 9:24AM,238616,20,Sartorius Stedim North America,1.0,NaN
4,Jun 2 2022 9:23AM,238615,12,Hush Hush,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 2 2022 9:43AM,238619,15,"Alliance Pharma, Inc.",11.0,0.0
1,Jun 2 2022 9:31AM,238618,20,"Andelyn Biosciences, Inc.",1.0,0.0
2,Jun 2 2022 9:27AM,238617,10,"Senseonics, Incorporated",0.0,1.0
3,Jun 2 2022 9:24AM,238616,20,Sartorius Stedim North America,1.0,0.0
4,Jun 2 2022 9:23AM,238615,12,Hush Hush,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1431629,23.0,5.0
12,1192870,71.0,0.0
15,238619,11.0,0.0
16,715603,2.0,1.0
19,954344,37.0,35.0
20,3101662,49.0,4.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1431629,23.0,5.0
1,12,1192870,71.0,0.0
2,15,238619,11.0,0.0
3,16,715603,2.0,1.0
4,19,954344,37.0,35.0
5,20,3101662,49.0,4.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,23.0,5.0
1,12,71.0,0.0
2,15,11.0,0.0
3,16,2.0,1.0
4,19,37.0,35.0
5,20,49.0,4.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,23.0
1,12,Released,71.0
2,15,Released,11.0
3,16,Released,2.0
4,19,Released,37.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20
Status,,,,,,
Executing,5.0,0.0,0.0,1.0,35.0,4.0
Released,23.0,71.0,11.0,2.0,37.0,49.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20
0,Executing,5.0,0.0,0.0,1.0,35.0,4.0
1,Released,23.0,71.0,11.0,2.0,37.0,49.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20
0,Executing,5.0,0.0,0.0,1.0,35.0,4.0
1,Released,23.0,71.0,11.0,2.0,37.0,49.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()